In [ ]:
# Import necessary libraries
import os
import subprocess
import tempfile
import time

# Define the paths to the required files
vcf_file = "/mnt/data/resources/gnomad/vep_gnomad_v4_hg19_exomes/gnomad.exomes.v4.1.sites.grch37.trimmed_liftover_norm_1e-1.bcf"
reference_file = "/mnt/data/resources/reference/ucsc/hg19_canonical.fa.gz"
chr_add_file = "~/projects/vcfcache/resources/chr_add.txt"

# Verify the files exist
print(f"VCF file exists: {os.path.exists(vcf_file)}")
print(f"Reference file exists: {os.path.exists(reference_file)}")
print(f"Chr add file exists: {os.path.exists(os.path.expanduser(chr_add_file))}")

# Define a function to run a command and capture output
def run_command(cmd, shell=False):
    try:
        if shell:
            result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
        else:
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"Command failed with exit code {e.returncode}")
        print(f"STDOUT: {e.stdout}")
        print(f"STDERR: {e.stderr}")
        return None

# Test extracting reference chromosomes
print("\nExtracting reference chromosomes:")
ref_cmd = f"samtools faidx {reference_file} | cut -f1 | head -5"
ref_chroms = run_command(ref_cmd, shell=True)
print(ref_chroms)

# Test querying variants
print("\nAttempting to query variants from VCF:")
query_cmd = f"bcftools query -f '%CHROM\\t%POS\\t%ID\\t%REF\\t%ALT\\n' {vcf_file} | head -5"
try:
    variants = run_command(query_cmd, shell=True)
    print(variants)
except Exception as e:
    print(f"Exception during variant query: {e}")

In [ ]:
# Check VCF format with bcftools
print("\nChecking VCF format:")
check_cmd = f"bcftools view -h {vcf_file} | head -5"
header = run_command(check_cmd, shell=True)
print(header)

# Check if VCF is indexed
print("\nChecking if VCF is indexed:")
index_cmd = f"ls -la {vcf_file}.csi"
index_result = run_command(index_cmd, shell=True)
if index_result:
    print("Index file exists")
else:
    print("Index file does not exist")

In [ ]:
# Test processing one variant
print("\nTesting variant processing:")
if 'variants' in locals() and variants:
    # Take the first variant
    variant_line = variants.strip().split('\n')[0]
    print(f"Processing variant: {variant_line}")
    
    # Extract chromosome and position
    fields = variant_line.split("\t")
    chrom = fields[0]
    pos = fields[1]
    ref_allele = fields[3]
    
    print(f"Chromosome: {chrom}, Position: {pos}, Ref Allele: {ref_allele}")
    
    # Try mapping chromosome
    chr_add_expanded = os.path.expanduser(chr_add_file)
    chr_map = {}
    with open(chr_add_expanded) as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 2:
                chr_map[parts[0]] = parts[1]
    
    ref_chrom = chr_map.get(chrom, chrom)
    print(f"Mapped chromosome: {ref_chrom}")
    
    # Try fetching reference sequence
    faidx_cmd = f"samtools faidx {reference_file} {ref_chrom}:{pos}-{int(pos) + len(ref_allele) - 1}"
    print(f"Running command: {faidx_cmd}")
    
    try:
        ref_seq = run_command(faidx_cmd, shell=True)
        print(f"Reference sequence result:")
        print(ref_seq)
        
        # Process the reference sequence to get just the bases
        if ref_seq:
            ref_bases = ''.join(ref_seq.strip().split('\n')[1:])
            print(f"Extracted bases: {ref_bases}")
            
            # Compare with ref allele
            print(f"Match: {ref_bases.lower() == ref_allele.lower()}")
    except Exception as e:
        print(f"Exception during reference lookup: {e}")

In [ ]:
# Check chromosome naming compatibility
print("\nComparing chromosome naming in VCF and reference:")

# Extract all chromosome names from reference
ref_chroms_cmd = f"samtools faidx {reference_file} | cut -f1"
ref_chroms_full = run_command(ref_chroms_cmd, shell=True)
ref_chroms_set = set(ref_chroms_full.strip().split('\n'))
print(f"Reference chromosomes: {', '.join(list(ref_chroms_set)[:5])}... (total: {len(ref_chroms_set)})")

# Extract unique chromosome names from VCF (first 100 variants)
vcf_chroms_cmd = f"bcftools query -f '%CHROM\\n' {vcf_file} | head -100 | sort | uniq"
vcf_chroms = run_command(vcf_chroms_cmd, shell=True)
if vcf_chroms:
    vcf_chroms_set = set(vcf_chroms.strip().split('\n'))
    print(f"VCF chromosomes: {', '.join(list(vcf_chroms_set)[:5])}... (from first 100 variants)")

    # Apply chromosome mapping
    mapped_vcf_chroms = {chr_map.get(chrom, chrom) for chrom in vcf_chroms_set}
    
    # Check if mapped VCF chromosomes exist in reference
    missing_chroms = mapped_vcf_chroms - ref_chroms_set
    if missing_chroms:
        print(f"WARNING: These chromosomes in VCF don't match reference after mapping: {missing_chroms}")
    else:
        print("All mapped VCF chromosomes exist in reference")

In [ ]:
# Test the validate_vcf_ref.sh script with debugging
print("\nTesting validate_vcf_ref.sh script with debugging:")

# Create a temporary script file with added debugging
with tempfile.NamedTemporaryFile(mode='w', suffix='.sh', delete=False) as temp_script:
    temp_script_path = temp_script.name
    
    # Add debugging to the validate_vcf_ref.sh script
    temp_script.write("""#!/bin/bash
set -e
set -o pipefail
set -x  # Enable debugging

if [ "$#" -ne 3 ]; then
    echo "Usage: $0 <vcf_file> <reference_fasta> <chr_add_file>"
    exit 1
fi

vcf_file="$1"
reference_fasta="$2"
chr_add_file="$3"

echo "VCF file: $vcf_file"
echo "Reference: $reference_fasta"
echo "Chr add file: $chr_add_file"

# Create a temporary file for validation results
tmp_file=$(mktemp)
echo "Using temp file: $tmp_file"

# Extract sample variants for validation
echo "Extracting variants..."
bcftools query -f '%CHROM\\t%POS\\t%REF\\n' "$vcf_file" | head -50 > "$tmp_file"
echo "Extracted $(wc -l < "$tmp_file") variants for validation"

# Validate each variant
echo "Starting validation..."
while IFS=$'\\t' read -r chrom pos ref_allele; do
    echo "Processing: